In [1]:
# jupyter中使用hyperparams需要添加
import sys
sys.argv = ['run.py']

根据经纬度映射出区域号.
- xlim:经度范围
- ylim:纬度范围

北京：
- lng_ld = 116.1
- lat_ld = 39.7
- lng_max = 116.7
- lat_max = 40.2

In [2]:
def get_gridID(longitude, latitude):
    # xlim = [116.1, 116.7]
    # ylim = [39.7, 40.2]
    lng_ld = 116.1
    lat_ld = 39.7
    lng_max = 116.7
    lat_max = 40.2
    grid_length = 0.0002
    grid_width = 0.0002
    # regionID = (xlim[1] - xlim[0]) // grid_length * (latitude - ylim[0]) // grid_width + (
    #         longitude - xlim[0]) // grid_length + 1
    gridID = str(int((longitude-lng_ld)/0.0002)) + "-" + str(int((latitude-lat_ld)/0.0002))
    return gridID

print(get_gridID(116.2345,39.8744))


672-871


轨迹过滤：过滤掉timeslots小于m的轨迹，过滤掉总条数少于5条的轨迹。

In [3]:
import os
file_list = sorted(os.listdir('./data/Geolife/Data'))
file_num = len(file_list)
file_list = file_list[1:]


for user_id in file_list:
    u_path = os.getcwd() + "/data" + "/Geolife" + "/Data/" + user_id + "/Trajectory"



# path = os.getcwd() + "/data" + "/Geolife" + "/Data" + "/001" + "/Trajectory"
# print(path)
# user_path = os.scandir(path)
# print(user_path)

In [4]:
output_path = './Results/size%d' % (1)
if not os.path.exists(output_path):
    # 目录不存在，进行创建操作
    os.makedirs(output_path)

  

In [5]:
import pandas as pd
import datetime
time0 = datetime.datetime.now()
start_time = datetime.datetime.now()
result_tmp = []
data_tmp = pd.DataFrame()
data_all = pd.DataFrame()  
# data_all.columns=['UID','timestamp','longitude','latitude']
for filename in file_list:
    path = "./data" + "/Geolife" + "/Data/" + filename + "/Trajectory"
    # user_path: one user's all trajectories
    user_path = os.scandir(path)
    for item in user_path:
        # path_item: one of the trajactories
        path_item = path + "/" + item.name
        with open(path_item, 'r+') as fp:
            # every item is a point
            # lat, lnt, 0, alt, date, date(日期), time（时间）
            # 39.999383,116.326916,0,134,39915.3152083333,2009-04-12,07:33:54
            for item in fp.readlines()[6::10]:
                item_list = item.strip().split(',')

                uid = filename
                lat = item_list[0]
                lnt = item_list[1]
                date = item_list[5]
                timestamp = item_list[6]

                temp = []
                temp.append(uid)
                temp.append(lat)
                temp.append(lnt)
                temp.append(date)
                temp.append(timestamp)
                result_tmp.append(temp) 
end_time = datetime.datetime.now()
print(len(result_tmp))
print("Loading Time:", str((end_time - start_time))[0:7])

2496075
Loading Time: 0:00:14


In [6]:
data_all = pd.DataFrame(result_tmp, columns=['uid', 'latitude', 'longitude', 'date', 'time']).dropna(axis=0)

tmp = data_all.copy(deep=True)
# 经纬度
# UID
tmp['uid'] = tmp['uid'].apply(lambda x: int(x))
# 获取日期
# 获取具体时间（24小时制）
# 获取时间编号（48个半小时中的第几个）
tmp['timeNo'] = tmp['time'].apply(lambda x: int(x[0:2]) * 2 + int(x[3:5]) // 30)
# 获得区域编号
tmp['gridID'] = tmp.apply(lambda x: get_gridID(float(x['longitude']), float(x['latitude'])), axis=1)
# tmp

```
['000', '40.000017', '116.327479', '2009-04-12', '07:33:03']
['000', '39.999383', '116.326916', '2009-04-12', '07:33:54']
['000', '39.998139', '116.327237', '2009-04-12', '07:34:44']
['000', '39.996225', '116.32658', '2009-04-12', '07:35:39']
['000', '39.995637', '116.326712', '2009-04-12', '07:36:24']
```

In [7]:
# 给区域重新编号
print("reindex regionID...")
keys = []
keys += set(tmp['gridID'])
# keys = [i for n, i in enumerate(keys) if i not in keys[:n]]
print("After loading the data files, region numbers: ", len(keys))
values = [i for i in range(1, len(keys) + 1)]
tmp_dict = dict(zip(keys, values))
tmp['regionID'] = tmp.apply(lambda x: tmp_dict[x['gridID']], axis=1)
print(tmp.shape)
tmp.head(5)

reindex regionID...
After loading the data files, region numbers:  732861
(2496075, 8)


,uid,latitude,longitude,date,time,timeNo,gridID,regionID
0,0,40.000017,116.327479,2009-04-12,07:33:03,15,1137-1500,179581
1,0,39.999383,116.326916,2009-04-12,07:33:54,15,1134-1496,517285
2,0,39.998139,116.327237,2009-04-12,07:34:44,15,1136-1490,380840
3,0,39.996225,116.32658,2009-04-12,07:35:39,15,1132-1481,652488
4,0,39.995637,116.326712,2009-04-12,07:36:24,15,1133-1478,47243


In [8]:
# 转换成AttnMove的数据格式：{uid, date, trajectory}
print("generate raw_pos.csv...")
tmp3 = tmp[['uid', 'date', 'regionID', 'timeNo']].copy(deep=True)

# aggregate the regionID for every time intervel(30 mins)
# tmp3 = tmp3.groupby(['uid', 'date', 'timeNo']).apply(lambda x: x[:])#.agg(lambda x: x.value_counts().index[0]).reset_index()
tmp3 = tmp3.groupby(['uid', 'date', 'timeNo']).agg(lambda x: x.value_counts().index[0]).reset_index()
tmp3.to_csv('./Results/tmp3.txt', sep='\t', index=False)
tmp3

generate raw_pos.csv...


,uid,date,timeNo,regionID
0,0,2008-10-23,5,510639
1,0,2008-10-23,6,443230
2,0,2008-10-23,8,614392
3,0,2008-10-23,9,311767
4,0,2008-10-23,19,70221
...,...,...,...,...
82543,181,2008-02-17,8,660717
82544,181,2008-02-17,21,403447
82545,181,2008-03-14,5,408919
82546,181,2008-03-14,6,700147


In [9]:
# 获得user_list和date_list
tmp3 = pd.read_csv('Results/tmp3.txt', sep="\t").groupby(['uid', 'date']).apply(lambda x: x[:])
user_list = tmp3['uid'].drop_duplicates().values
date_list = tmp3['date'].drop_duplicates().values
print(user_list)
print(date_list)

[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181]
['2008-10-23' '2008-10-24' '2008-10-26' ... '2008-03-23' '2008-03-25'
 '2008-02-06']


In [10]:
# result_final是符合AttnMove数据格式的dataframe
result_final = pd.DataFrame()
no_use = 0
for uid in user_list:
    for date in date_list:
        try:
            tmp4 = pd.DataFrame(tmp3.loc[(uid, date), ['timeNo', 'regionID']].values.T)
            tmp4 = tmp4.set_axis(tmp4.iloc[0], axis=1, inplace=False).drop(index=0)
            tmp4.index = pd.MultiIndex.from_product([[uid], [date]])
            result_final = result_final.append(tmp4)
        except:
            # print("No record for UID=%d, date=%s" % (uid, date))
            no_use = no_use + 1
            continue

# 没有出现的时段列也补上相应的列
seq_len = 48
column_names = result_final.columns
for index in range(0,seq_len):
    if index not in column_names:
        result_final.insert(index, index, None)

result_final = result_final.fillna('*')  

print(result_final.shape)

(11142, 48)


In [11]:
# 运行一次太慢了，因此跑完之后存下来方便之后直接用
result_final.to_csv(output_path + '/raw_pos.csv', index=True, header=True)
df_tmp = pd.read_csv(output_path + '/raw_pos.csv', index_col=0)
df_tmp.to_csv(output_path + '/raw_pos.txt', sep='\t', index=True, header=True)
